In [1]:
import pandas as pd
import requests
from datetime import datetime
from jsonpath import jsonpath


def get_zt_stock_bank(date: str = None) -> pd.DataFrame:
    """
    获取指定日期涨停股票行情

    Parameters
    ----------
    date : str
        指定日期，默认为当前日期 格式形如 ``'20220506'``

    Returns
    -------
    DataFrame
        指定日期涨停股票行情

    Fields
    ------
    ``['日期', '股票代码', '股票名称', '涨跌幅', '成交额', '封板资金', '流通市值', '最新价', '换手率','首次封板时间', '最后封板时间', '炸板次数', '连扳数', '所属行业']``
    """
    if date is None:
        date = datetime.today().strftime('%Y%m%d')
    params = (
        ('ut', 'b2884a393a59ad64002292a3e90d46a5'),
        ('pn', '1'),
        ('pz', '500'),
        ('po', '1'),
        ('np', '1'),
        ('fields', 'f12,f13,f14,f62'),
        ('fid','f62'),
        ('fs','m:90+t:2'),
    )

    response = requests.get(
        'https://push2.eastmoney.com/api/qt/clist/get', params=params,  verify=False)
    fields = {
        'f12': '板块编码',
        'f13': '上市板块',
        'f14': '板块名称',
        'f62': '主力资金净流入'
    }
    items = jsonpath(response.json(), '$..diff[:]')
    if not items:
        df = pd.DataFrame(
            columns=['日期']+list(fields.values())+['统计天数', '涨停次数'])
        return df
    df = pd.DataFrame(items)
    
    
    df: pd.DataFrame = df.rename(columns=fields)[fields.values()]
    
    df['主力资金净流入'] /= 100000000
    
    return df

df1 = get_zt_stock_bank()
df1

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'push2.eastmoney.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,板块编码,上市板块,板块名称,主力资金净流入
0,BK0464,90,石油行业,6.458748
1,BK1019,90,化学原料,5.847164
2,BK0437,90,煤炭行业,3.654532
3,BK0740,90,教育,3.620175
4,BK0477,90,酿酒行业,3.588133
...,...,...,...,...
81,BK0428,90,电力行业,-5.286962
82,BK0910,90,专用设备,-5.455277
83,BK1032,90,风电设备,-5.577269
84,BK0451,90,房地产开发,-6.630099


In [31]:
import pandas as pd 
import numpy as np

df1 = pd.DataFrame({'depth': [0.500000,0.600000,1.300000],'VAR1': [38.196202,38.198002,38.200001],'profile': ['profile_1','profile_1','profile_1']})
df2 = pd.DataFrame({'depth': [0.600000,1.100000,1.200000],'VAR2': [0.20440,0.20442,0.20446]})
df3 = pd.DataFrame({'depth': [1.200000,1.300000,1.400000],'VAR3': [15.1880,15.1820,15.1820]})
df1
df1

,depth,VAR1,profile
0,0.5,38.196202,profile_1
1,0.6,38.198002,profile_1
2,1.3,38.200001,profile_1


In [6]:
# dfmi['one']['second']
dfmi.loc[:, ('one', 'second')]

0    b
1    f
2    j
3    n
Name: (one, second), dtype: object

In [4]:

import pandas as pd
import numpy as np

idx = pd.MultiIndex.from_arrays([
    ['warm', 'warm', 'cold', 'cold'],
    ['dog', 'falcon', 'fish', 'spider']],
    names=['blooded', 'animal'])
s = pd.Series([4, 2, 0, 8], name='legs', index=idx)
s

blooded  animal
warm     dog       4
         falcon    2
cold     fish      0
         spider    8
Name: legs, dtype: int64